In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb

import data_utils as data


tt = gp.Model('IIITB Exams Timetable')

1002
[(55, 57), (7, 17), (26, 30), (7, 26), (7, 35), (48, 54), (21, 37), (6, 57), (10, 27), (41, 42), (10, 36), (25, 43), (3, 6), (44, 56), (3, 24), (14, 24), (14, 33), (14, 51), (15, 16), (47, 48), (17, 44), (18, 21), (29, 36), (48, 49), (21, 32), (6, 43), (29, 45), (2, 27), (25, 29), (2, 36), (2, 45), (22, 32), (13, 51), (3, 28), (16, 56), (55, 56), (7, 16), (5, 46), (5, 55), (17, 57), (6, 29), (29, 31), (6, 38), (6, 56), (33, 37), (10, 35), (10, 44), (25, 42), (51, 54), (14, 32), (5, 41), (28, 43), (6, 24), (9, 57), (29, 35), (29, 44), (6, 42), (21, 40), (39, 48), (20, 53), (2, 26), (13, 32), (43, 45), (24, 41), (35, 41), (32, 54), (13, 50), (16, 46), (16, 55), (28, 29), (5, 27), (9, 25), (5, 36), (47, 51), (5, 45), (6, 10), (17, 56), (29, 30), (6, 46), (2, 3), (10, 25), (24, 27), (32, 40), (3, 4), (24, 36), (35, 36), (35, 45), (24, 45), (13, 54), (9, 29), (6, 14), (6, 23), (42, 44), (39, 47), (2, 7), (12, 39), (20, 52), (12, 48), (4, 44), (23, 57), (13, 22), (24, 31), (43, 44), (13

## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
def exam_length(course:int):
    return 120

def exam_slots(course: int):
    # return int(exam_length(course)//15)
    return int(data.get_course_duration(course)//15)

exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [3]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [4]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [5]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
                          for c in range(data.n_courses)
                          for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
              for d in range(data.n_days)
              for r in range(data.n_rooms)
              for t in range(data.n_times)),
              name = "room_capacity_constraint"
             )
tt.update()


In [6]:
# Constraint 3: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    for c in range(data.n_courses)
)

tt.update()


In [7]:
# Constraint 4: Student clash constraint

tt.addConstrs((gp.quicksum(E[c, b, e, d] for c in data.intersection_pairs[i] for b, e in covering_sessions(c, t)) <= 1
               for i in range(data.n_intersection_pairs)
               for d in range(data.n_days)
               for t in range(data.n_times)),
              name='student_clash_constraint'
             )

tt.update()

In [8]:
# Constraint 5: No course can have more than 3 rooms alloted

M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

D1 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D1',
)

D2 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D2',
)

for c,r in M:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
    tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
    tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
    tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
    tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')

tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

In [9]:
# Constraint 6: Lab exams are conducted in labs

tt.addConstrs(
    (
        gp.quicksum(C[c, r] for r in (data.get_not_rooms_course(c)) )== 0
        for c in range(data.n_courses)
    ),
    name="lab_exam_constraint",
)

tt.update()

Course: 0
Rooms: [4, 5, 18]
Course: 1
Rooms: [4, 5, 18]
Course: 2
Rooms: [4, 5, 18]
Course: 3
Rooms: [4, 5, 18]
Course: 4
Rooms: [4, 5, 18]
Course: 5
Rooms: [4, 5, 18]
Course: 6
Rooms: [4, 5, 18]
Course: 7
Rooms: [4, 5, 18]
Course: 8
Rooms: [4, 5, 18]
Course: 9
Rooms: [4, 5, 18]
Course: 10
Rooms: [4, 5, 18]
Course: 11
Rooms: [4, 5, 18]
Course: 12
Rooms: [4, 5, 18]
Course: 13
Rooms: [4, 5, 18]
Course: 14
Rooms: [4, 5, 18]
Course: 15
Rooms: [4, 5, 18]
Course: 16
Rooms: [4, 5, 18]
Course: 17
Rooms: [4, 5, 18]
Course: 18
Rooms: [4, 5, 18]
Course: 19
Rooms: [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Course: 20
Rooms: [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Course: 21
Rooms: [4, 5, 18]
Course: 22
Rooms: [4, 5, 18]
Course: 23
Rooms: [4, 5, 18]
Course: 24
Rooms: [4, 5, 18]
Course: 25
Rooms: [4, 5, 18]
Course: 26
Rooms: [4, 5, 18]
Course: 27
Rooms: [4, 5, 18]
Co

In [10]:
# Constraint 7: No triplet courses should be scheduled in the day

tt.addConstrs(
    (
        (E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) + E.sum(c3, '*', '*', d) <= 2)
        for c1, c2, c3 in data.intersection_triplets
        for d in range(data.n_days)
    ),
    name="triplet_constraint",
)


{(1, 18, 32, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18, 32, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18, 32, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18, 32, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18, 32, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18, 32, 5): <gurobi.Constr *Awaiting Model Update*>,
 (7, 8, 56, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 8, 56, 1): <gurobi.Constr *Awaiting Model Update*>,
 (7, 8, 56, 2): <gurobi.Constr *Awaiting Model Update*>,
 (7, 8, 56, 3): <gurobi.Constr *Awaiting Model Update*>,
 (7, 8, 56, 4): <gurobi.Constr *Awaiting Model Update*>,
 (7, 8, 56, 5): <gurobi.Constr *Awaiting Model Update*>,
 (6, 29, 36, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 29, 36, 1): <gurobi.Constr *Awaiting Model Update*>,
 (6, 29, 36, 2): <gurobi.Constr *Awaiting Model Update*>,
 (6, 29, 36, 3): <gurobi.Constr *Awaiting Model Update*>,
 (6, 29, 36, 4): <gurobi.Constr *Awaiting Model Update*>,
 (6, 29, 36, 5): <gu

## Soft Constraints

In [11]:
# # 1. No 2 exams on a day for a student

# # minimize summation courseintersection(c1, c2) * ((E[c1, *, *, d] + E[c2, *, *, d]) > 1) for all c1, c2, d


# obj = 0
# for i in range(data.n_intersection_pairs):
#     c1, c2 = data.intersection_pairs[i]
#     weight = data.get_intersection_of_courses(c1, c2)
#     for d in range(data.n_days):
#         obj += weight*gp.and_((E.sum(c1, '*', '*', d) and E.sum(c2, '*', '*', d)))



# make a slack variable for each day for every intersection pair X[c1, c2, d]. Will be 1 if on same day and 0 if not
X = tt.addVars(
    [
        (c1, c2, d)
        for c1, c2 in data.intersection_pairs
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="X",
)

tt.addConstrs(
   (X[c1, c2, d] == (E.sum(c1, '*', '*', d) and E.sum(c2, '*', '*', d))
    for c1, c2 in data.intersection_pairs
    for d in range(data.n_days)
   ),
    name="soft_constraint1"
)

tt.update()


## Objective Function

In [12]:
# obj is summation of X[c1, c2, d] * weight(c1, c2) for all c1, c2, d
def get_intersection_of_courses(c1, c2):
    return (set(data.course_student[c1]) & set(data.course_student[c2]))
obj1 = gp.quicksum(X[c1, c2, d]*len(get_intersection_of_courses(c1, c2))
                         for c1, c2 in data.intersection_pairs for d in range(data.n_days))


In [13]:
# Soft Constraint 2: minimize the number of rooms used
obj2 = gp.quicksum(M[c,r] for c in range(data.n_courses) for r in range(data.n_rooms))



In [14]:
print(data.times[16], data.times[25])
obj3 = gp.quicksum(E[c, b, e, d] for c in range(data.n_courses) for b, e, d in data.get_preferred_slots(c))
# for c in range(data.n_courses):
#     for b, e, d in data.get_preferred_slots(c):
#         print("c: ", c, "b: ", b, "e: ", e, "d: ", d)
#         obj3 += E[c, b, e, d]

12:00 15:00


In [15]:
objective = 13*obj1 + 7*obj2 - 20*obj3
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [16]:
import time


def cb(model, where):
    if where == GRB.Callback.MIPNODE:
        # Get model objective
        obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

        # Has objective changed?
        if abs(obj - model._cur_obj) > 1e-8:
            # If so, update incumbent and time
            model._cur_obj = obj
            model._time = time.time()

    # Terminate if objective has not improved in 60s
    if time.time() - model._time > 60:
        model.terminate()

tt._cur_obj = float('inf')
tt._time = time.time()

tt.optimize(callback=cb)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04 LTS")

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2616227 - for non-commercial use only - registered to dh___@iiitb.ac.in
Optimize a model with 108366 rows, 15520 columns and 1197188 nonzeros
Model fingerprint: 0xc6d1a568
Model has 7068 quadratic constraints
Variable types: 0 continuous, 15520 integer (13722 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e+00, 4e+03]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 3e+02]
  QRHS range       [6e+00, 2e+02]
Presolve removed 90380 rows and 6036 columns (presolve time = 5s)...
Presolve removed 99224 rows and 6563 columns (presolve time = 10s)...
Presolve removed 99224 rows and 6563 columns (presolve time = 16s)...
Presolve removed 98996 rows and 65

In [17]:
E_sol = tt.getAttr('x', E)
C_sol = tt.getAttr('x', C)
M_sol = tt.getAttr('x', M)
D1_sol = tt.getAttr('x', D1)
D2_sol = tt.getAttr('x', D2)
solution=[]
for i in E_sol:
    if E_sol[i] == 1:
        solution.append(i)

GurobiError: Unable to retrieve attribute 'x'

In [18]:
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

In [19]:
! pip install tabulate
! pip install matplotlib

In [20]:
for c,b,e,d in solution:
    l = []
    for(r) in range(data.n_rooms):
        if C_sol[c,r] > 0:
            l.append((data.get_room_num(r), C_sol[c,r]))
    print(f"Course {data.course_id_code[c]} has {data.get_course_strength(c)} and is scheduled in slot {data.times[b]} to {data.times[e]} on day {data.days[d]} in room {l}")

Course AIM 101  has 59 and is scheduled in slot 14:45 to 17:45 on day Wed in room [('R102', 59.0)]
Course AIM 102  has 59 and is scheduled in slot 08:45 to 11:45 on day Mon in room [('A310', 59.0)]
Course AIM 704  has 20 and is scheduled in slot 08:45 to 11:45 on day Mon in room [('A204', 20.0)]
Course AIM 821  has 21 and is scheduled in slot 14:45 to 17:45 on day Fri in room [('R103', 21.0)]
Course AIM 825  has 158 and is scheduled in slot 16:30 to 18:00 on day Thu in room [('R103', 158.0)]
Course AIM 829  has 132 and is scheduled in slot 14:15 to 17:15 on day Sat in room [('R401(Amanthran)', 132.0)]
Course AIM 831  has 175 and is scheduled in slot 08:00 to 11:00 on day Thu in room [('A106', 175.0)]
Course AIM 832  has 93 and is scheduled in slot 10:00 to 13:00 on day Sat in room [('R401(Amanthran)', 93.0)]
Course AIM 843  has 35 and is scheduled in slot 14:15 to 17:15 on day Mon in room [('A106', 35.0)]
Course AIM 845  has 7 and is scheduled in slot 14:45 to 16:45 on day Fri in room 

In [21]:
from collections import defaultdict
from tabulate import tabulate

schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri","Sat"]

print("EXAM TIMETABLE")
print("=" * 50)
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            del entry["start"]
        print(f"\n{day}")
        print("-" * 50)
        print(tabulate(sorted_entries, headers="keys", tablefmt="grid"))

EXAM TIMETABLE

Mon
--------------------------------------------------
+----------+------------+---------------+---------------------------------------------------+
| Course   |   Strength | Time          | Rooms                                             |
+==========+============+===============+===================================================+
| AIM 704  |         20 | 08:45 - 11:45 | A204 (Capacity: 20.0)                             |
+----------+------------+---------------+---------------------------------------------------+
| AIM 102  |         59 | 08:45 - 11:45 | A310 (Capacity: 59.0)                             |
+----------+------------+---------------+---------------------------------------------------+
| EGC 301  |        193 | 08:45 - 11:45 | A106 (Capacity: 193.0)                            |
+----------+------------+---------------+---------------------------------------------------+
| DHS 108  |          7 | 09:15 - 12:15 | A308 (Capacity: 7.0)                     

In [22]:
## Verification of the solution

# 1 - No student should have two exams at the same time

def time_to_minutes(time_str):
    """Converts a HH:MM string to minutes since midnight."""
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# Build a dictionary that maps each course (c) to its one exam slot details.
# Assume each course appears only once in the solution.
course_slots = {}
for c, b, e, d in solution:
    # Save day and exam start/end times (converted to minutes) for each course
    day = data.days[d]
    start = time_to_minutes(data.times[b])
    end   = time_to_minutes(data.times[e])
    course_slots[c] = (day, start, end)

# Check for any student conflicts between every pair of distinct courses.
clash_found = False
course_indices = list(course_slots.keys())
for i in range(len(course_indices)):
    c1 = course_indices[i]
    day1, start1, end1 = course_slots[c1]
    for j in range(i + 1, len(course_indices)):
        c2 = course_indices[j]
        day2, start2, end2 = course_slots[c2]
        # Check if courses are on the same day and if their times overlap:
        if day1 == day2 and (start1 < end2 and start2 < end1):
            # Check intersection of enrolled students; get_intersection_of_courses returns set of common student IDs.
            common_students = set(data.course_student[c1]) & set(data.course_student[c2])
            if common_students:
                clash_found = True
                print(f"Clash found between Course {data.course_id_code[c1]} and Course {data.course_id_code[c2]}")
                print("Common student(s):", common_students)
                print(f"--> {data.course_id_code[c1]}: {data.times[b]} - {data.times[e]} ;",  # later you can format times properly
                      f"{data.course_id_code[c2]}: {data.times[b]} - {data.times[e]}\n")

if not clash_found:
    print("No student clashes found!")


# 2 - The total number of students writing an exam in a room should not exceed the room capacity


room_violations = False

# Process each room separately
for r in range(data.n_rooms):
    # Build a schedule per day for room r.
    room_schedule = defaultdict(list)
    for c, b, e, d in solution:
        allocated = C_sol[c, r]
        if allocated > 0:
            day = data.days[d]
            start = time_to_minutes(data.times[b])
            end = time_to_minutes(data.times[e])
            course_code = data.course_id_code[c]
            room_schedule[day].append((start, end, allocated, course_code))
    
    # For each day, process the exam intervals for room r.
    for day, intervals in room_schedule.items():
        events = []
        for start, end, alloc, course in intervals:
            # When exam starts, add the number of students;
            # when exam ends, subtract that number.
            events.append((start, alloc))
            events.append((end, -alloc))
        events.sort(key=lambda x: x[0])
        
        current_alloc = 0
        for time, change in events:
            current_alloc += change
            if current_alloc > data.get_room_capacity(r):
                room_violations = True
                print(f"Room capacity violation on {day} in room {data.get_room_num(r)}:")
                print(f"   Room capacity: {data.room_capacity[r]}, allocated: {current_alloc}")
                break  # Report once per day per room

if not room_violations:
    print("No room capacity violations found!")


No student clashes found!
No room capacity violations found!


In [23]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")

Student 1 has exams for courses [('AIM 101 ', '14:45', '17:45', 'Wed'), ('AIM 102 ', '08:45', '11:45', 'Mon'), ('CSE 102 ', '15:00', '18:00', 'Mon'), ('CSE 102 P -B', '10:30', '12:30', 'Tue'), ('CSE 104 ', '10:00', '13:00', 'Sat'), ('DAS 101 ', '08:00', '11:00', 'Thu'), ('DAS 101P -B', '08:00', '09:30', 'Tue'), ('DHS 101B ', '08:00', '10:00', 'Fri'), ('DHS 201 ', '09:00', '12:00', 'Wed')]
Student 2 has exams for courses [('AIM 101 ', '14:45', '17:45', 'Wed'), ('AIM 102 ', '08:45', '11:45', 'Mon'), ('CSE 102 ', '15:00', '18:00', 'Mon'), ('CSE 102 P -B', '10:30', '12:30', 'Tue'), ('CSE 104 ', '10:00', '13:00', 'Sat'), ('DAS 101 ', '08:00', '11:00', 'Thu'), ('DAS 101P -B', '08:00', '09:30', 'Tue'), ('DHS 101B ', '08:00', '10:00', 'Fri'), ('DHS 201 ', '09:00', '12:00', 'Wed')]
Student 3 has exams for courses [('AIM 101 ', '14:45', '17:45', 'Wed'), ('AIM 102 ', '08:45', '11:45', 'Mon'), ('CSE 102 ', '15:00', '18:00', 'Mon'), ('CSE 102 P -B', '10:30', '12:30', 'Tue'), ('CSE 104 ', '10:00', '

In [24]:
for s in range(data.n_students):
    print(f"Student {s+1} has exam on: " , [data.days[d] for c,b,e,d in solution if E_sol[c,b,e,d] == 1 and c in data.get_courses_student(s)])

Student 1 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 2 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 3 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 4 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 5 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 6 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 7 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 8 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 9 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 10 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 11 has exam on:  ['Wed', 'Mon', 'Mon', 'Tue', 'Sat', 'Thu', 'Tue', 'Fri', 'Wed']
Student 12 has exam on:  ['Wed

In [25]:
# print how many courses have got the preferred slots
for c in range(data.n_courses):
    print(f"Course {data.course_id_code[c]} has got the preferred slots: ", [data.times[b] for b,e,d in data.get_preferred_slots(c) if E_sol[c,b,e,d] == 1])

Course AIM 101  has got the preferred slots:  ['14:45']
Course AIM 102  has got the preferred slots:  ['08:45']
Course AIM 704  has got the preferred slots:  ['08:45']
Course AIM 821  has got the preferred slots:  ['14:45']
Course AIM 825  has got the preferred slots:  ['16:30']
Course AIM 829  has got the preferred slots:  ['14:15']
Course AIM 831  has got the preferred slots:  ['08:00']
Course AIM 832  has got the preferred slots:  ['10:00']
Course AIM 843  has got the preferred slots:  ['14:15']
Course AIM 845  has got the preferred slots:  ['14:45']
Course AIM 846  has got the preferred slots:  ['09:00']
Course AMS 401  has got the preferred slots:  ['08:30']
Course AMS 402  has got the preferred slots:  ['14:00']
Course AMS 403  has got the preferred slots:  ['08:30']
Course AMS 404  has got the preferred slots:  ['15:00']
Course COM 605  has got the preferred slots:  ['08:15']
Course COM 837  has got the preferred slots:  ['08:15']
Course COM 863  has got the preferred slots:  ['